## Import Libraries

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Define the NN model

In [33]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, 7),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Dropout(0.2),
            nn.Conv2d(32, 50, 3),
            nn.ReLU(),
            nn.BatchNorm2d(50),
            # nn.MaxPool2d(kernel_size = 2, stride = 2),
            # nn.Dropout(0.25),
            nn.Conv2d(50, 32, 1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            # nn.Dropout(0.25)
        )
        self.fc1 = nn.Linear(32, 10)
        # self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(F.adaptive_avg_pool2d(x, (1, 1)))
        x = x.view(-1, 32)
        # x = F.relu(self.fc1(x))
        x = self.fc1(x)
        return F.log_softmax(x)

### Create the model object and print the summary

In [34]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 22, 22]           1,600
              ReLU-2           [-1, 32, 22, 22]               0
       BatchNorm2d-3           [-1, 32, 22, 22]              64
         MaxPool2d-4           [-1, 32, 11, 11]               0
           Dropout-5           [-1, 32, 11, 11]               0
            Conv2d-6             [-1, 50, 9, 9]          14,450
              ReLU-7             [-1, 50, 9, 9]               0
       BatchNorm2d-8             [-1, 50, 9, 9]             100
            Conv2d-9             [-1, 32, 9, 9]           1,632
             ReLU-10             [-1, 32, 9, 9]               0
      BatchNorm2d-11             [-1, 32, 9, 9]              64
           Linear-12                   [-1, 10]             330
Total params: 18,240
Trainable params: 18,240
Non-trainable params: 0
---------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Define the train and test loaders

In [38]:
torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)),
                        transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                        # transforms.ColorJitter(brightness=0.2, contrast=0.2)
                        # transforms.Normalize((0.485,), (0.229,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        # transforms.Normalize((0.485,), (0.229,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

## Define the train and test loop

In [39]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct_train = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct_train += pred.eq(target.view_as(pred)).sum().item()
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        loss, correct_train, len(train_loader.dataset),
        100. * correct_train / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [40]:
model = Net().to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

for epoch in range(20):
    print("epoch:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch: 0


loss=0.13611803948879242 batch_id=234: 100%|██████████| 235/235 [00:46<00:00,  5.03it/s]


Train set: Average loss: 0.1361, Accuracy: 52280/60000 (87.13%)




Test set: Average loss: 0.0901, Accuracy: 9714/10000 (97.14%)

epoch: 1


loss=0.03646126762032509 batch_id=234: 100%|██████████| 235/235 [00:47<00:00,  4.97it/s]

...


Train set: Average loss: 0.0220, Accuracy: 59082/60000 (98.47%)




Test set: Average loss: 0.0247, Accuracy: 9922/10000 (99.22%)

epoch: 19


loss=0.03002290427684784 batch_id=234: 100%|██████████| 235/235 [00:47<00:00,  4.96it/s]


Train set: Average loss: 0.0300, Accuracy: 59108/60000 (98.51%)




Test set: Average loss: 0.0232, Accuracy: 9926/10000 (99.26%)



epoch: 16
loss=0.099638931453228 batch_id=234: 100%|██████████| 235/235 [00:47<00:00,  4.95it/s]
Train set: Average loss: 0.0996, Accuracy: 59091/60000 (98.48%)